In [ ]:
#######################################################################################################################################################################
# Angle Calculation using Mediapipe Pose Visualization #
#######################################################################################################################################################################
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    """Calculate angle between three 2D points using arctangent function."""
    ba = np.array(a) - np.array(b)
    bc = np.array(c) - np.array(b)

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))

    return np.degrees(angle)

# Open webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to RGB
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        # Define key joints (Using Mediapipe landmark indices)
        joint_ids = {
            "nose": 0,
            "left_shoulder": 11, "right_shoulder": 12,
            "left_elbow": 13, "right_elbow": 14,
            "left_wrist": 15, "right_wrist": 16,
            "left_waist": 23, "right_waist": 24  # Waist is hip landmarks
        }

        # Extract 2D coordinates
        joints = {}
        for name, idx in joint_ids.items():
            if landmarks[idx].visibility > 0.5:  # Ensure the joint is visible
                joints[name] = (int(landmarks[idx].x * frame.shape[1]), 
                                int(landmarks[idx].y * frame.shape[0]))

        # Compute mid_shoulder (midpoint of left and right shoulder)
        if all(k in joints for k in ["left_shoulder", "right_shoulder"]):
            joints["mid_shoulder"] = ((joints["left_shoulder"][0] + joints["right_shoulder"][0]) // 2,
                                      (joints["left_shoulder"][1] + joints["right_shoulder"][1]) // 2)

        # Compute angles only if required points exist
        angles = {}
        if all(k in joints for k in ["left_shoulder", "left_elbow", "left_wrist"]):
            angles["Left Elbow"] = calculate_angle(joints["left_shoulder"], joints["left_elbow"], joints["left_wrist"])
        if all(k in joints for k in ["right_shoulder", "right_elbow", "right_wrist"]):
            angles["Right Elbow"] = calculate_angle(joints["right_shoulder"], joints["right_elbow"], joints["right_wrist"])
        if all(k in joints for k in ["left_elbow", "left_shoulder", "left_waist"]):
            angles["Left Arm Lift"] = calculate_angle(joints["left_elbow"], joints["left_shoulder"], joints["left_waist"])
        if all(k in joints for k in ["right_elbow", "right_shoulder", "right_waist"]):
            angles["Right Arm Lift"] = calculate_angle(joints["right_elbow"], joints["right_shoulder"], joints["right_waist"])
        if all(k in joints for k in ["nose", "mid_shoulder", "left_shoulder"]):
            angles["Neck Angle"] = calculate_angle(joints["nose"], joints["mid_shoulder"], joints["left_shoulder"])

        # Draw key points and lines
        for (p1, p2, p3, label, text_pos) in [
            ("left_shoulder", "left_elbow", "left_wrist", "Left Elbow", "left_elbow"),
            ("right_shoulder", "right_elbow", "right_wrist", "Right Elbow", "right_elbow"),
            ("left_elbow", "left_shoulder", "left_waist", "Left Arm Lift", "left_shoulder"),
            ("right_elbow", "right_shoulder", "right_waist", "Right Arm Lift", "right_shoulder")
        ]:
            if all(k in joints for k in [p1, p2, p3]):
                # Draw lines
                cv2.line(frame, joints[p1], joints[p2], (0, 255, 0), 3)
                cv2.line(frame, joints[p2], joints[p3], (0, 255, 0), 3)

                # Draw key points
                cv2.circle(frame, joints[p1], 6, (0, 0, 255), -1)
                cv2.circle(frame, joints[p2], 6, (255, 0, 0), -1)
                cv2.circle(frame, joints[p3], 6, (0, 255, 255), -1)

                # Display angle
                if label in angles:
                    angle_text = f"{label}: {int(angles[label])}°"
                    cv2.putText(frame, angle_text, (joints[text_pos][0] + 20, joints[text_pos][1]), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # Draw neck angle separately
        if all(k in joints for k in ["nose", "mid_shoulder", "left_shoulder"]) and "Neck Angle" in angles:
            cv2.line(frame, joints["nose"], joints["mid_shoulder"], (0, 255, 0), 3)
            cv2.line(frame, joints["mid_shoulder"], joints["left_shoulder"], (0, 255, 0), 3)

            cv2.circle(frame, joints["nose"], 6, (0, 0, 255), -1)
            cv2.circle(frame, joints["mid_shoulder"], 6, (255, 0, 0), -1)
            cv2.circle(frame, joints["left_shoulder"], 6, (0, 255, 255), -1)

            # Display neck angle
            angle_text = f"Neck Angle: {int(angles['Neck Angle'])}°"
            cv2.putText(frame, angle_text, (joints["nose"][0] + 20, joints["nose"][1]), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # Show the frame
    cv2.imshow("Pose Angle Visualization", frame)

    # Exit if 'q' is pressed
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
#######################################################################################################################################################################
# Move Detection, Recording and Preprocessing #
#######################################################################################################################################################################

import cv2
import mediapipe as mp
import json
import numpy as np
import time
import os

# Load timeline.json
with open("timeline.json", "r") as f:
    timeline_data = json.load(f)

# Extract moves
moves = timeline_data["moves"]

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Directory to save processed reference data
output_dir = "reference_moves"
os.makedirs(output_dir, exist_ok=True)

def calculate_angle(a, b, c):
    """Calculate angle between three 3D points using the arctangent function."""
    ba = np.array(a) - np.array(b)
    bc = np.array(c) - np.array(b)

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    
    return np.degrees(angle)

def process_frame(image):
    """Extract pose landmarks from a frame and compute angles."""
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        
        # Extract key joint coordinates (normalized values)
        joint_ids = {
            "nose": 0,
            "left_shoulder": 11, "right_shoulder": 12,
            "left_elbow": 13, "right_elbow": 14,
            "left_wrist": 15, "right_wrist": 16,
            "left_hip": 23, "right_hip": 24
        }
        
        joints = {name: (landmarks[idx].x, landmarks[idx].y, landmarks[idx].z) for name, idx in joint_ids.items()}

        # Compute mid-shoulder point
        if "left_shoulder" in joints and "right_shoulder" in joints:
            mid_shoulder = (
                (joints["left_shoulder"][0] + joints["right_shoulder"][0]) / 2,
                (joints["left_shoulder"][1] + joints["right_shoulder"][1]) / 2,
                (joints["left_shoulder"][2] + joints["right_shoulder"][2]) / 2
            )
        else:
            mid_shoulder = None

        # Compute angles
        angles = {}

        if all(k in joints for k in ["left_shoulder", "left_elbow", "left_wrist"]):
            angles["left_elbow"] = calculate_angle(joints["left_shoulder"], joints["left_elbow"], joints["left_wrist"])
        if all(k in joints for k in ["right_shoulder", "right_elbow", "right_wrist"]):
            angles["right_elbow"] = calculate_angle(joints["right_shoulder"], joints["right_elbow"], joints["right_wrist"])
        if all(k in joints for k in ["left_elbow", "left_shoulder", "left_hip"]):
            angles["left_arm_lift"] = calculate_angle(joints["left_elbow"], joints["left_shoulder"], joints["left_hip"])
        if all(k in joints for k in ["right_elbow", "right_shoulder", "right_hip"]):
            angles["right_arm_lift"] = calculate_angle(joints["right_elbow"], joints["right_shoulder"], joints["right_hip"])
        if mid_shoulder and "nose" in joints:
            angles["neck_angle"] = calculate_angle(joints["nose"], mid_shoulder, joints["left_shoulder"])

        return angles
    
    return None

# Open webcam
cap = cv2.VideoCapture(0)

for move in moves:
    move_name = move["name"]
    duration = move["duration"]
    
    print(f"Get ready for move: {move_name}")
    time.sleep(2)  # Give user time to prepare
    
    print(f"Recording move: {move_name} (duration: {duration}s)...")
    
    start_time = time.time()
    capture_time = start_time + (duration / 2)  # Capture a frame at the middle of the move
    
    captured_frame = None

    while time.time() - start_time < duration:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame!")
            continue
        
        if time.time() >= capture_time and captured_frame is None:
            captured_frame = frame.copy()
    
    if captured_frame is not None:
        angles = process_frame(captured_frame)
        if angles:
            # Save angles for this move
            with open(os.path.join(output_dir, f"{move_name}.json"), "w") as f:
                json.dump(angles, f, indent=4)
            print(f"Saved reference angles for {move_name}")
        else:
            print(f"Failed to detect pose for {move_name}")

cap.release()
cv2.destroyAllWindows()
